The previous experiment had poor results b/c the vectors were too sparse. We'll try again by making a lexicon from all epigrams (not just the Antipaters). Try to get something denser or at least more meaninful. Also will lemmatize.

In [59]:
import json
import unicodedata

In [25]:
from cltk import NLP

In [26]:
nlp = NLP("grc")

‎𐤀 CLTK version '1.1.6'.
Pipeline for language 'Ancient Greek' (ISO: 'grc'): `GreekNormalizeProcess`, `GreekStanzaProcess`, `GreekEmbeddingsProcess`, `StopsProcess`.


In [27]:
doc = nlp.analyze("Χριστὸν παμβασιλῆα φίλοις καμάτοισι γεραίρων")

In [28]:
doc.words[4].lemma

'γέραιρος'

# Load Epigrams

In [29]:
with open("all_epigrams.json", "r") as fo:
    all_epigrams = json.load(fo)

In [30]:
# all_epigrams

# Get Greek sentences

Look through `all_epigrams` and make a list of strings, the strings being the greek epigram text.

In [31]:
greek_sentences = []

In [32]:
for epigram in all_epigrams:
    for epigram_text in epigram["texts"]:
        if epigram_text["language"] == "grc":
            greek_sentences.append(epigram_text['text'])

# Get both Antipaters' epigrams

In [42]:
all_texts = list()
for epigram in all_epigrams:
    #print(epigram.keys())
    texts = epigram["texts"]
    for text_obj in texts:
        if text_obj["language"] == "grc":
            text = text_obj["text"]
            all_texts.append(text)
print(len(all_texts))

4218


In [44]:
all_texts[:5]

['\n                      ἃς οἱ πλάνοι καθεῖλον ἐνθάδ᾽ εἰκόνας\n ἄνακτες ἐστήλωσαν εὐσεβεῖς πάλιν.\n',
 '\n                      θεῖος Ἰουστῖνος, Σοφίης πόσις, ᾧ πόρε Χριστὸς\n πάντα διορθοῦσθαι, καὶ κλέος ἐν πολέμοις,\nμητρὸς ἀπειρογάμοιο δόμον σκάζοντα νοήσας,\n σαθρὸν ἀποσκεδάσας τεῦξέ μιν ἀσφαλέως.\n',
 '\n                     ὁ πρὶν Ἰουστῖνος περικαλλέα δείματο νηὸν\nτοῦτον μητρὶ Θεοῦ, κάλλεϊ λαμπόμενον:\nὁπχότερος δὲ μετ᾽ αὐτὸν Ἰουστῖνος βασιλεύων\nκρείσσονα τῆς προτέρης ὤπασεν ἀγλαΐην.\n\n                  ',
 '\n                      τοῦτον Ἰωάννῃ, Χριστοῦ μεγάλῳ θεράποντι,\n Στούδιος ἀγλαὸν οἶκον ἐδείματο: καρπαλίμως δὲ\n τῶν κάμεν εὕρετο μισθόν, ἑλὼν ὑπατηίδα ῥάβδον.\n',
 '\n                     τόνδε Θεῷ κάμες οἶκον, Ἀμάντιε, μεσσόθι πόντου,\nτοῖς πολυδινήτοις κύμασι μαρνάμενος.\nοὐ νότος, οὐ βορέης ἱερὸν σέο δῶμα τινάξει,\n νηῷ θεσπεσίῳ τῷδε φυλασσόμενον.\n\n                      ζώοις ἤματα πολλά: σὺ γὰρ νεοθηλέα Ῥώμην,\n πόντῳ ἐπαΐξας, θήκαο φαιδροτέρην.\n']

In [45]:
all_antipater = {'sidon':[], 'thess':[], 'dubious':[]}  

antipater_thess_id = '3'
antipater_sid_id = '17'

for epigram in all_epigrams:
    poem = dict()
    
    for author in epigram['authors']:
        if author['url'] == 'https://anthologiagraeca.org/api/authors/'+antipater_thess_id+'/':
            if 'author' in poem:
                poem['author'] ='dubious'
            else: 
                poem['author'] = 'thess'
                
        if author['url'] == 'https://anthologiagraeca.org/api/authors/'+antipater_sid_id+'/':
            if 'author' in poem:
                poem['author']  ='dubious'
            else: 
                poem['author'] = 'sidon'
                
    if 'author' in poem:
        poem['book'] = epigram["book"]["number"]
        poem['poem_num'] = epigram["fragment"]
        for epigram_text in epigram["texts"]:
            if epigram_text["language"] == "grc":
                poem['text']= epigram_text['text']
        all_antipater[poem['author'] ].append(poem);
    

In [46]:
all_antipater.keys()

dict_keys(['sidon', 'thess', 'dubious'])

In [47]:
len(all_antipater["sidon"])

86

In [48]:
len(all_antipater["thess"])

97

In [49]:
len(all_antipater['dubious'])

1

In [50]:
all_antipater["sidon"][0]

{'author': 'sidon',
 'book': 6,
 'poem_num': 10,
 'text': 'Τριτογενές, Σώτειρα, Διὸς φυγοδέμνιε κούρα,\n Παλλάς, ἀπειροτόκου δεσπότι παρθενίης,\nβωμόν τοι κεραοῦχον ἐδείματο τόνδε Σέλευκος,\n Φοιβείαν ἰαχὰν φθεγγομένου στόματος.'}

In [51]:
texts_sidon = [_dict["text"] for _dict in all_antipater["sidon"]]

In [52]:
texts_thess = [_dict["text"] for _dict in all_antipater["thess"]]

In [53]:
text_dubious = [_dict["text"] for _dict in all_antipater["dubious"]]

# Preprocess

In [54]:
texts_sidon[0]

'Τριτογενές, Σώτειρα, Διὸς φυγοδέμνιε κούρα,\n Παλλάς, ἀπειροτόκου δεσπότι παρθενίης,\nβωμόν τοι κεραοῦχον ἐδείματο τόνδε Σέλευκος,\n Φοιβείαν ἰαχὰν φθεγγομένου στόματος.'

In [79]:
# !pip install tqdm
from tqdm.notebook import trange

In [84]:
def clean_texts(list_of_texts):
    """Clean punctuation, lemmatize w/ cltk"""
    clean_texts = list()
    for idx in trange(len(list_of_texts)):
        text = list_of_texts[idx]
        clean_text = text.replace(",", "")
        clean_text = clean_text.replace(".", "")
        clean_text = clean_text.replace(":", "")
        clean_text = clean_text.replace("\n", " ")
        clean_text = clean_text.replace("᾽", "") #TODO: rm this if lemmatizing
        cltk_doc = nlp.analyze(clean_text)
        clean_text = " ".join([word.lemma for word in cltk_doc.words])
        #print(dir(cltk_doc))
        #input()
        #clean_text = unicodedata.normalize("NFKD", clean_text).translate({ord(c): None for c in "̓̔́̀͂̈ͅ"})
        clean_texts.append(clean_text)
    return clean_texts

In [85]:
clean_all_epigrams = clean_texts(all_texts)

  0%|          | 0/4218 [00:00<?, ?it/s]

In [86]:
len(clean_all_epigrams)

4218

In [87]:
clean_all_epigrams[:2]

['ὅς ὁ πλάνος καθαιρέω ἔνος εἰκών ἀνακτής στηλόω εὐσεβής πάλιν',
 'θεῖος Ἰουστῖς Σοφίας πόσις ὅς πόρα Χριστός πᾶς διορθόω καί κλέος ἐν πόλεμος μήτηρ ἀπειρόγαμος δόμος σκάζω νοέω σαθρός ἀποσκεδάννυμι τυγχάνω μιν ἀσφαλέως']

In [88]:
clean_sidon = clean_texts(list_of_texts=texts_sidon)

  0%|          | 0/86 [00:00<?, ?it/s]

In [89]:
clean_thess = clean_texts(list_of_texts=texts_thess)

  0%|          | 0/97 [00:00<?, ?it/s]

In [90]:
clean_dubious = clean_texts(list_of_texts=text_dubious)

  0%|          | 0/1 [00:00<?, ?it/s]

In [91]:
len(clean_sidon)

86

In [92]:
len(clean_thess)

97

In [93]:
len(clean_dubious)

1

In [94]:
clean_sidon[0]

'Τριτογενής Σώτειρα Ζεύς φυγοδεμνίζω κούρα Παλλάς ἀπειρότοκος δεσπότις παρθενία βωμός σύ κεραοῦχος δείμνυμι ὅδε Σέλευκος Φοιβεία ἰαχά φθέγγομαι στόμα'

# Count Words (etc.)

In [95]:
from sklearn.feature_extraction.text import CountVectorizer

In [96]:
epigram_count_vectorizer = CountVectorizer(min_df=2, max_features=2000)

In [98]:
epigram_count_vectorizer.fit(all_texts)

CountVectorizer(max_features=2000, min_df=2)

In [100]:
len(epigram_count_vectorizer.vocabulary_)

2000

In [101]:
vectors_sidon = epigram_count_vectorizer.transform(clean_sidon)

In [102]:
vectors_thess = epigram_count_vectorizer.transform(clean_thess)

In [103]:
vector_dubious = epigram_count_vectorizer.transform(clean_dubious)

# Compare similarity

In [104]:
# from sklearn.metrics.pairwise import cosine_similarity

In [105]:
# from sklearn.metrics.pairwise import cosine_distances

In [106]:
from scipy.spatial.distance import cosine

In [107]:
cosine([1, 0, 0], [0, 1, 0])

1.0

In [108]:
cosine([1, 0, 0], [0, 1, 0])

1.0

In [109]:
from scipy.spatial.distance import euclidean

In [139]:
def calculate_average_similarity(one_vector, many_vectors):
    distance_sum = 0
    for idx, vector in enumerate(many_vectors):
        vector_array = vector.toarray()[0]
        vector_dubious_array = one_vector.toarray()[0]
        sim = cosine(vector_array, vector_dubious_array)
        distance_sum += float(sim)
    avg = distance_sum / many_vectors.shape[0]
    return avg

In [140]:
calculate_average_similarity(one_vector=vector_dubious, many_vectors=vectors_sidon)

0.7127476097407357

In [141]:
calculate_average_similarity(one_vector=vector_dubious, many_vectors=vectors_thess)

0.7313254458955518

# Compare the dubious to all

Print the 10 most similar epigrams

In [142]:
from sklearn.metrics.pairwise import cosine_distances

In [144]:
dubius_cf_sidon = cosine_distances(vector_dubious.todense(), vectors_sidon.todense())

C:\Users\willi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\willi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [148]:
dubius_cf_thess = cosine_distances(vector_dubious.todense(), vectors_thess.todense())

C:\Users\willi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(
C:\Users\willi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\utils\validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [149]:
dubius_cf_sidon.shape

(1, 86)

In [150]:
dubius_cf_sidon

array([[0.80304035, 0.73197187, 0.72888615, 0.67223933, 0.5847726 ,
        0.62095098, 0.53576165, 0.75229246, 0.75185417, 0.59940363,
        0.702225  , 0.78335544, 0.73912057, 0.7923863 , 0.7452281 ,
        0.6060807 , 0.62153458, 0.8607285 , 0.58548913, 0.58894585,
        0.9241902 , 0.68416192, 0.87950495, 0.7452281 , 0.51289198,
        0.66096825, 1.        , 0.71673559, 0.89278875, 0.63606987,
        0.81430466, 0.89472064, 0.69876238, 0.78034315, 0.5976601 ,
        0.64368659, 0.56222773, 0.6696483 , 0.88628529, 0.43932679,
        1.        , 0.82383393, 0.8607285 , 0.86869357, 0.52833337,
        0.71290733, 0.58894585, 0.58599566, 0.59528639, 0.68481246,
        0.84236793, 0.66561188, 0.72005374, 0.539447  , 0.57019881,
        0.78990971, 0.90715233, 0.72286735, 0.70865718, 0.45758337,
        0.62095098, 1.        , 0.66437569, 1.        , 0.77021374,
        0.76770895, 0.72203072, 0.93219365, 0.63582148, 0.80892107,
        0.95357617, 0.73197187, 0.83918312, 1.  

In [151]:
dubius_cf_thess

array([[0.82492476, 0.72703958, 0.61504057, 0.86869357, 0.65955855,
        0.8607285 , 0.78913367, 0.63606987, 0.82492476, 0.7287746 ,
        0.70638989, 0.68416192, 0.57019881, 1.        , 0.82383393,
        0.83918312, 0.90446054, 0.81430466, 0.61830686, 0.9241902 ,
        0.61830686, 0.67149085, 0.77021374, 1.        , 0.86002687,
        0.64766787, 0.8511125 , 0.8245344 , 0.75086356, 0.4736032 ,
        0.68416192, 0.64766787, 0.6083911 , 0.85319495, 0.78557749,
        0.8880215 , 0.87543178, 0.64968863, 0.58304608, 0.67173392,
        0.71338161, 0.67702888, 0.55925872, 0.61373035, 0.55897357,
        0.88538635, 0.61397568, 0.71925504, 0.73466568, 0.93810155,
        0.61669003, 0.70934082, 0.57981941, 0.65955855, 0.78115595,
        0.63234199, 0.62706507, 0.78149289, 0.78557749, 0.47889375,
        0.5847726 , 0.62095098, 0.78335544, 0.68416192, 0.69676078,
        0.88122886, 0.90566667, 0.72145699, 0.96040962, 0.82492476,
        0.82492476, 0.9241902 , 0.76788083, 0.58

In [152]:
dubius_cf_thess.shape

(1, 97)

In [153]:
import numpy as np

In [168]:
# https://numpy.org/doc/stable/reference/generated/numpy.argsort.html
sidon_idx = np.argsort(dubius_cf_sidon)
sidon_idx_descending = np.flip(sidon_idx)

In [169]:
sidon_idx

array([[39, 59, 80, 74, 24, 44,  6, 53, 82, 36, 54, 85,  4, 18, 47, 19,
        46, 48, 34,  9, 15, 75, 60,  5, 16, 68, 29, 35, 77, 25, 62, 51,
        37,  3, 21, 49, 84, 32, 10, 58, 45, 27, 52, 66, 57,  2, 81,  1,
        71, 12, 14, 23,  8,  7, 65, 64, 33, 11, 79, 55, 78, 13,  0, 69,
        30, 41, 72, 50, 76, 42, 17, 43, 22, 38, 28, 31, 56, 20, 67, 70,
        61, 40, 26, 83, 73, 63]], dtype=int64)

In [170]:
sidon_idx_descending

array([[63, 73, 83, 26, 40, 61, 70, 67, 20, 56, 31, 28, 38, 22, 43, 17,
        42, 76, 50, 72, 41, 30, 69,  0, 13, 78, 55, 79, 11, 33, 64, 65,
         7,  8, 23, 14, 12, 71,  1, 81,  2, 57, 66, 52, 27, 45, 58, 10,
        32, 84, 49, 21,  3, 37, 51, 62, 25, 77, 35, 29, 68, 16,  5, 60,
        75, 15,  9, 34, 48, 46, 19, 47, 18,  4, 85, 54, 36, 82, 53,  6,
        44, 24, 74, 80, 59, 39]], dtype=int64)

In [196]:
# https://numpy.org/doc/stable/reference/generated/numpy.argsort.html
thess_idx = np.argsort(dubius_cf_thess)
thess_idx_descending = np.flip(thess_idx)

In [197]:
thess_idx

array([[29, 59, 76, 83, 44, 42, 74, 12, 91, 86, 75, 52, 38, 60, 96, 73,
        78, 32, 43, 46,  2, 50, 18, 20, 61, 56, 55,  7, 31, 25, 37, 87,
        53,  4, 77, 21, 39, 41, 79, 30, 11, 63, 64, 84, 10, 51, 40, 47,
        93, 67,  1,  9, 48, 85, 28, 94, 72, 22, 54, 82, 57, 62, 58, 34,
         6, 81, 92, 17, 90, 14, 27,  0,  8, 70, 69, 15, 88, 26, 33, 24,
         5,  3, 36, 65, 45, 35, 95, 16, 66, 19, 71, 49, 80, 68, 89, 13,
        23]], dtype=int64)

In [198]:
thess_idx_descending

array([[23, 13, 89, 68, 80, 49, 71, 19, 66, 16, 95, 35, 45, 65, 36,  3,
         5, 24, 33, 26, 88, 15, 69, 70,  8,  0, 27, 14, 90, 17, 92, 81,
         6, 34, 58, 62, 57, 82, 54, 22, 72, 94, 28, 85, 48,  9,  1, 67,
        93, 47, 40, 51, 10, 84, 64, 63, 11, 30, 79, 41, 39, 21, 77,  4,
        53, 87, 37, 25, 31,  7, 55, 56, 61, 20, 18, 50,  2, 46, 43, 32,
        78, 73, 96, 60, 38, 52, 75, 86, 91, 12, 74, 42, 44, 83, 76, 59,
        29]], dtype=int64)

In [199]:
dubius_cf_thess[0][63]

0.6841619171453817

In [201]:
print("Unknown 'Antipater' epigram:")
print(text_dubious[0])
print("*" * 80)
print("Most similar epigrams of Antipater of Sidon:")
print("*" * 80)
for idx, sim_idx in enumerate(sidon_idx_descending[0][:10]):
    distance = dubius_cf_sidon[0][sim_idx]
    sim_text = texts_sidon[sim_idx]
    print(f"{idx}.\nSimilarity score: {distance}\n{sim_text}\n\n")

Unknown 'Antipater' epigram:
Τανταλὶς ἅδε ποχ᾽ ἁ δίσς1᾽ ἑπτάκι τέκνα τεκοῦσα
 γαστρὶ μιῇ, Φοίβῳ θῦμα καὶ Ἀρτέμιδι:

                   κούρα γὰρ προὔπεμψε κόραις φόνον, ἄρρεσι δ᾽ ἄρσην
 δισσοὶ γὰρ δισσὰς ἔκτανον ἑβδομάδας.

                  ἁ δὲ τόσας ἀγέλας μάτηρ πάρος, ἁ πάρος εὔπαις,
οὐδ᾽ ἐφ᾽ ἑνὶ τλάμων λείπετο γηροκόμῳ:
 μάτηρ δ᾽ οὐχ ὑπὸ παισίν, ὅπερ θέμις, ἀλλ᾽ ὑπὸ ματρὸς
 παῖδες ἐς ἀλγεινοὺς πάντες ἄγοντο τάφους.
 Τάνταλε, καὶ δὲ σὲ γλῶσσα διώλεσε, καὶ σέο κούραν

                  χἀ μὲν ἐπετρώθη, σοὶ δ᾽ ἔπι δεῖμα λίθος.
********************************************************************************
Most similar epigrams of Antipater of Sidon:
********************************************************************************
0.
Similarity score: 1.0
Μναμοσύναν ἕλε θάμβος, ὅτ᾽ ἔκλυε τᾶς μελιφώνου
Σαπφοῦς,  μὴ δεκάταν Μοῦσαν ἔχουσι βροτοί.


1.
Similarity score: 1.0
τὰν δάμαλιν, βουφορβέ, παρέρχεο, μηδ᾽ ἀπάνευθε
 συρίσδῃς: μαστῷ πόρτιν ἀπεκδέχεται.


2.
Similarity score: 1.0
τὰν ἀ

In [202]:
print("Unknown 'Antipater' epigram:")
print(text_dubious[0])
print("*" * 80)
print("Most similar epigrams of Antipater of Thessalonica:")
print("*" * 80)
for idx, sim_idx in enumerate(thess_idx_descending[0][:10]):
    distance = dubius_cf_thess[0][sim_idx]
    sim_text = texts_thess[sim_idx]
    print(f"{idx}.\nSimilarity score: {distance}\n{sim_text}\n\n")

Unknown 'Antipater' epigram:
Τανταλὶς ἅδε ποχ᾽ ἁ δίσς1᾽ ἑπτάκι τέκνα τεκοῦσα
 γαστρὶ μιῇ, Φοίβῳ θῦμα καὶ Ἀρτέμιδι:

                   κούρα γὰρ προὔπεμψε κόραις φόνον, ἄρρεσι δ᾽ ἄρσην
 δισσοὶ γὰρ δισσὰς ἔκτανον ἑβδομάδας.

                  ἁ δὲ τόσας ἀγέλας μάτηρ πάρος, ἁ πάρος εὔπαις,
οὐδ᾽ ἐφ᾽ ἑνὶ τλάμων λείπετο γηροκόμῳ:
 μάτηρ δ᾽ οὐχ ὑπὸ παισίν, ὅπερ θέμις, ἀλλ᾽ ὑπὸ ματρὸς
 παῖδες ἐς ἀλγεινοὺς πάντες ἄγοντο τάφους.
 Τάνταλε, καὶ δὲ σὲ γλῶσσα διώλεσε, καὶ σέο κούραν

                  χἀ μὲν ἐπετρώθη, σοὶ δ᾽ ἔπι δεῖμα λίθος.
********************************************************************************
Most similar epigrams of Antipater of Thessalonica:
********************************************************************************
0.
Similarity score: 1.0

                     ἀνθέα τὸν ναυηγὸν ἐπὶ στόμα Πηνειοῖο
 νυκτὸς ὑπὲρ βαιῆς νηξάμενον σανίδος,
 μούνιος ἐκ θάμνοιο θορὼν λύκος, ἄσκοπον ἄνδρα,
 ἔκτανεν. ὦ γαίης κύματα πιστότερα,

                  


1.
Similarity score: 1.